In [0]:
#Step 1 - Read the CSV file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import current_timestamp 

In [0]:

races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [0]:
races_df = spark.read.csv("dbfs:/FileStore/races.csv", schema =races_schema,header=True )

In [0]:
#Step 2 - Add ingestion date and race_timestamp to the dataframe

In [0]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [0]:
races_with_timestamp_df = races_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')).withColumn("ingestion_date",current_timestamp())

In [0]:
#Step 3 - Select only the columns required & rename as required

In [0]:
races_selected_df = races_with_timestamp_df .select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

In [0]:
#Write the output to processed container in parquet format

In [0]:
races_selected_df.write.mode("overwrite").parquet("dbfs:/FileStore/Formula1/processed/f1_processed_races") 

In [0]:
races_selected_df.show(truncate=False)

+-------+---------+-----+----------+---------------------+-----------------------+-------------------+
|race_id|race_year|round|circuit_id|name                 |ingestion_date         |race_timestamp     |
+-------+---------+-----+----------+---------------------+-----------------------+-------------------+
|1      |2009     |1    |1         |Australian Grand Prix|2024-04-23 04:06:46.352|2009-03-29 06:00:00|
|2      |2009     |2    |2         |Malaysian Grand Prix |2024-04-23 04:06:46.352|2009-04-05 09:00:00|
|3      |2009     |3    |17        |Chinese Grand Prix   |2024-04-23 04:06:46.352|2009-04-19 07:00:00|
|4      |2009     |4    |3         |Bahrain Grand Prix   |2024-04-23 04:06:46.352|2009-04-26 12:00:00|
|5      |2009     |5    |4         |Spanish Grand Prix   |2024-04-23 04:06:46.352|2009-05-10 12:00:00|
|6      |2009     |6    |6         |Monaco Grand Prix    |2024-04-23 04:06:46.352|2009-05-24 12:00:00|
|7      |2009     |7    |5         |Turkish Grand Prix   |2024-04-23 04:0